In [6]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

In [7]:
# 设置超参数
num_epochs = 10
batch_size = 32
learning_rate = 0.001

# 数据增强和预处理
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [8]:
# 数据加载
data_dir = './dataset_element_recognized/0'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=batch_size,
                             shuffle=True, num_workers=4)
               for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [9]:
# 定义模型
model_ft = models.resnet50(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, len(class_names))

model_ft = model_ft.to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_ft.parameters(), lr=learning_rate, momentum=0.9)

# 训练模型
def train_model(model, criterion, optimizer, num_epochs=25):
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        print('-' * 10)

        # 每个epoch都有训练和验证阶段
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # 训练模式
            else:
                model.eval()   # 验证模式

            running_loss = 0.0
            running_corrects = 0

            # 迭代数据
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # 梯度清零
                optimizer.zero_grad()

                # 前向传播
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # 只有在训练阶段才进行反向传播和优化
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # 统计
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

    return model

/home/u2021213565/.conda/envs/myenv/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/u2021213565/.conda/envs/myenv/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /home/u2021213565/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:07<00:00, 13.5MB/s]


In [10]:
# 训练和验证模型
model_ft = train_model(model_ft, criterion, optimizer, num_epochs=num_epochs)

Epoch 1/10
----------
train Loss: 2.2637 Acc: 0.3188
val Loss: 2.3000 Acc: 0.2667
Epoch 2/10
----------
train Loss: 1.9083 Acc: 0.4783
val Loss: 2.4857 Acc: 0.2667
Epoch 3/10
----------
train Loss: 1.8295 Acc: 0.4783
val Loss: 2.7048 Acc: 0.2667
Epoch 4/10
----------
train Loss: 1.7788 Acc: 0.4783
val Loss: 2.5187 Acc: 0.2667
Epoch 5/10
----------
train Loss: 1.6351 Acc: 0.5072
val Loss: 2.4669 Acc: 0.2000
Epoch 6/10
----------
train Loss: 1.7212 Acc: 0.2899
val Loss: 2.3556 Acc: 0.2667
Epoch 7/10
----------
train Loss: 1.5746 Acc: 0.4203
val Loss: 2.2838 Acc: 0.2667
Epoch 8/10
----------
train Loss: 1.3293 Acc: 0.5797
val Loss: 2.2536 Acc: 0.2667
Epoch 9/10
----------
train Loss: 1.3392 Acc: 0.5072
val Loss: 2.1980 Acc: 0.2667
Epoch 10/10
----------
train Loss: 1.2752 Acc: 0.5072
val Loss: 2.1695 Acc: 0.2667


In [ ]:
# 保存模型
torch.save(model_ft.state_dict(), 'resnet50.pth')